<a href="https://colab.research.google.com/github/jdxxmahmud/Llama-Basic-Rag/blob/main/llamaindex_basic_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Requirements
!pip install --upgrade pip
!pip install llama-index-core==0.10.50
!pip install llama-index-readers-file==0.1.25
!pip install llama-index-llms-nvidia==0.1.3
!pip install llama-index-embeddings-nvidia==0.1.4
!pip install llama-index-postprocessor-nvidia-rerank==0.1.2
!pip install ipywidgets==8.1.3

  Using cached pip-24.2-py3-none-any.whl.metadata (3.6 kB)
Using cached pip-24.2-py3-none-any.whl (1.8 MB)
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.4/15.4 MB 35.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 46.3 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 29.7 MB/s eta 0:00:00
  Attempting uninstall: llama-index-core
    Found existing installation: llama-index-core 0.10.50
    Uninstalling llama-index-core-0.10.50:
      Successfully uninstalled llama-index-core-0.10.50
  Using cached jedi-0.19.1-py2.py3-none-any.whl.metadata (22 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 26.5 MB/s eta 0:00:00
Using cached jedi

In [51]:
# prompt: get the NVIDIA_API_KEY variable from secrets

from google.colab import userdata

nvidia_api_key = userdata.get('NVIDIA_API_KEY')
if not nvidia_api_key.startswith("nvapi-"):
    nvidia_api_key = input("Please enter your NVIDIA API key: ")

    assert nvidia_api_key.startswith("nvapi-"), f"{nvidia_api_key[:5]}... is not a valid key"

# print(nvidia_api_key)


## Initialize the LLM

*llama-index-llm-nvidia*, also known as NVIDIA's LLM connector, allows your connect to and generate from compatible models available on the NVIDIA API catalog



In [52]:
from llama_index.core import Settings
from llama_index.llms.nvidia import NVIDIA

Settings.llm = NVIDIA(model="mistralai/mixtral-8x7b-instruct-v0.1", nvidia_api_key=nvidia_api_key)

## Initialize the embedding

In [53]:
from llama_index.embeddings.nvidia import NVIDIAEmbedding
Settings.embed_model = NVIDIAEmbedding(model="NV-Embed-QA", truncate="END", nvidia_api_key=nvidia_api_key)

## Obtain some toy text dataset

In [54]:
import os
TOY_DATA_PATH = "./data/"

from llama_index.core.node_parser import SentenceSplitter
from llama_index.core import SimpleDirectoryReader
Settings.text_splitter = SentenceSplitter(chunk_size=400)
documents = SimpleDirectoryReader(TOY_DATA_PATH).load_data()

## Process the documents into VectorStoreIndex

In [55]:
from llama_index.core import VectorStoreIndex

index = VectorStoreIndex.from_documents(documents)

## Create a Query Engine to ask questions over my data

In [56]:
# Returns a Query engine for this index.
query_engine = index.as_query_engine(similarity_top_k=10)

In [57]:
response = query_engine.query(
    "Summarize the information in one sentence"
)
print(response)

In the new context, the focus is on maintaining health during winter, addressing potential issues like vitamin D deficiency, indoor allergens, and cold-related illnesses, while also providing advice for elderly care and winter healthcare tips.


In [59]:
response = query_engine.query(
    "Name the medicines mentioned in the document"
)
print(response)

The medicines mentioned in the document are:

1. Pseudoephedrine (e.g., Sudafed)
2. Diphenhydramine (e.g., Benadryl)
3. Loratadine (e.g., Claritin)
4. Dextromethorphan (e.g., Robitussin DM)
5. Guaifenesin (e.g., Mucinex)
6. Acetaminophen (e.g., Tylenol)
7. Ibuprofen (e.g., Advil)
8. Albuterol (Ventolin, ProAir)
9. Fluticasone (e.g., Flovent)
10. Amoxicillin or Azithromycin (Z-Pack)
11. Benzocaine lozenges (e.g., Cepacol)
12. Phenol spray (e.g., Chloraseptic)
13. Paracetamol (Acetaminophen)
14. Loratadine (e.g., Claritin) or Cetirizine (e.g., Zyrtec)
15. Fluticasone (e.g., Flonase)
16. Sertraline (e.g., Zoloft) or Fluoxetine (e.g., Prozac)
17. Petroleum jelly or Cetaphil cream
18. Beeswax balm or Petroleum-based balm
19. Oral Rehydration Salts (ORS)
20. Loperamide (e.g., Imodium)
21. Vitamin C supplements
22. Zinc lozenges
23. Beta-blockers, ACE inhibitors, or statins

Some medicines are repeated in the list as they are mentioned in different sections of the context. The context highlig

In [60]:
response = query_engine.query(
    "What should I do for backpain?"
)
print(response)

For back pain, you can try applying heat or cold therapy to the afflicted area to alleviate pain and inflammation. Over-the-counter pain relievers, like acetaminophen or ibuprofen, can also be beneficial. It's crucial to maintain good posture and follow proper lifting techniques to avoid additional strain on your back. If the discomfort continues or intensifies, seek advice from a healthcare professional for further assessment and treatment options. Regular exercise, comprising stretching and strengthening exercises, can help alleviate back pain and enhance overall back health.


## Enhancing accuracy for single data sources

In [63]:
# We will narrow the collection to 40 results and further narrow it to 4 with the reranker.
from llama_index.postprocessor.nvidia_rerank import NVIDIARerank

reranker_query_engine = index.as_query_engine(
    similarity_top_k=40, node_postprocessors=[NVIDIARerank(top_n=4, nvidia_api_key=nvidia_api_key)],
)

response = reranker_query_engine.query(
    "I have leg pain, what should I do?"
)
print(response)

Based on the symptoms you've described, it's possible that the cold weather may be contributing to your leg pain. There are a few things you can do to alleviate this discomfort.

First, consider applying heat to the affected area. This can help improve circulation and reduce muscle tension. A heating pad or a warm bath could provide some relief.

Second, make sure you're staying active. Regular exercise can help keep your muscles flexible and strong, which can reduce pain and prevent injury. If outdoor activities are not an option due to the weather, consider indoor exercises like yoga or home workouts.

Third, ensure you're wearing appropriate clothing and footwear for the cold weather. Dress in layers to keep warm, and choose moisture-wicking materials for your base layer to keep sweat away from your skin. For your feet, wear shoes that provide good support and traction to prevent slips and falls on icy surfaces.

If your leg pain persists or worsens, it would be best to consult with